In [5]:
import pandas as pd
import sys
import math
import bs4 as bs
import urllib.request
import re
import PyPDF2
import nltk
from nltk.stem import WordNetLemmatizer 
import spacy
#importing necessary modules
import os
import glob
import pdfplumber

### Prueba

In [6]:
#Execute this line if you are running this code for first time
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/codespace/nltk_data...


True

In [7]:
#Initializing few variable
nlp = spacy.load('en_core_web_lg')
lemmatizer = WordNetLemmatizer()

In [8]:
#Function to Read PDF File and return its Text
def pdfReader(pdf_path):
    
    with open(pdf_path, 'rb') as pdfFileObject:
        pdfReader = PyPDF2.PdfFileReader(pdfFileObject)
        count = pdfReader.numPages
        print("\nTotal Pages in pdf = ", count)
        
        c = 'Y'
        start_page = 0
        end_page = count-1
        c = input("Do you want to read entire pdf ?[Y]/N  :  ")
        if c == 'N' or c == 'n' :
            start_page  = int(input("Enter start page number (Indexing start from 0) :  "))
            end_page = int(input(f"Enter end page number (Less than {count}) : "))
            
            if start_page <0 or start_page >= count:
                print("\nInvalid Start page given")
                sys.exit()
                
            if end_page <0 or end_page >= count:
                print("\nInvalid End page given")
                sys.exit()
                
        for i in range(start_page,end_page+1):
            page = pdfReader.getPage(i)

        return page.extractText()

In [9]:
#Step 3. Getting Text 

input_text_type = int(input("Select one way of inputting your text  \
: \n1. Type your Text(or Copy-Paste)\n2. Load from .txt file\n3. Load from .pdf file\n4. From Wikipedia Page URL\n\n"))

if input_text_type == 1:
    text = input(u"Enter your text : \n\n")

elif input_text_type == 2:
    txt_path = input("Enter file path :  ")
    text = file_text(txt_path)
    
        
elif input_text_type == 3:
    file_path = input("Enter file path :  ")
    text = pdfReader(file_path)
    
elif input_text_type == 4:
    wiki_url = input("Enter Wikipedia URL to load Article : ")
    text = wiki_text(wiki_url)
    
else:
    print("Sorry! Wrong Input, Try Again.")

ValueError: invalid literal for int() with base 10: ''

### Extractor de entidades

In [ ]:
#assigning the NLP model
#https://spacy.io/usage/models
#you need to download the spacy model before starting this step. 
#More info on the website of spacy to download spacy model. 
#You can also load your own custom made model if you want

nlp_model=spacy.load("en_core_web_lg")

In [ ]:
#creation of a dataframe to include the columns 
df_PA4= pd.DataFrame(columns = ['Text No.','Entities', 'Labels'])
#input files path where your pdf files are
files_path= 'C:/Users/USUARIO/Desktop/tfm'
#reading the path
read_files=glob.glob(os.path.join(files_path,"*.pdf")) 
#looping over the pdf files
for files in read_files:
    #if the filename is clear and distinct
    from pathlib import Path
    #Extracting the basename of the file
    base = os.path.basename(files)
    #Extract the filename from the basename of the file
    filename=Path(base).stem
    #if the filename is not the official name  of the application
    #filename="DMP_"+str(i)
    text=''
    #using pdfplumber for processing the pdf files
    with pdfplumber.open(files) as pdf:
        for pdf_page in pdf.pages:
            single_page_text=pdf_page.extract_text()
            text=text+ '\n' + single_page_text
    tx="  ".join(text.split('\n'))
    #Load SpaCy Model
    docs=nlp_model(tx)
    #initialize entities and labels variables
    entities = []
    labels = []
    for ent in docs.ents:
        entities.append(ent)
        labels.append(ent.label_)
    #collecting the entities and the corresponding labels of the file in the loop in a dataframe
    df = pd.DataFrame({'Text No.':filename,'Entities':entities,'Labels':labels})
    #assigning all the datatypes within df as string
    df = df.astype('str')
    #dropping the duplicates
    df.drop_duplicates(subset=['Text No.','Entities','Labels'], keep="first", inplace=True)
    #appending data of the file in the loop 
    df_PA4 = df_PA4.append(df, ignore_index=True)
df_PA4.head()